In [18]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD, lr_scheduler
import shutil, sys   
import copy
import torch.nn as nn

import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy 

from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader



In [19]:
BATCH_SIZE = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

In [47]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        img, label = original_tuple
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (img, label, path)
        return tuple_with_path

In [48]:
def test_performance(model, test_loader, class_names):
        model.eval()
        running_loss = 0
        running_corrects = 0
        
        file1 = open("VGG_predictions_on_random-split-data.txt","a") 
        with torch.no_grad():
            
            number_test_steps = 0
            for batch_id, (inputs, labels, paths) in enumerate(test_loader):
                inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() # sum up loss of all minibatches
                number_test_steps += 1
                
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data).cpu()
                
                image_path = paths[0]
                image_name = image_path.split('/')[-1]
                string = image_name + "\t" + class_names[preds.item()] + "\n"
                print(string)
                file1.write(string)
            
            epoch_testloss = running_loss / number_test_steps
            epoch_testaccuracy = running_corrects.double() / len(test_loader.dataset)
            
            print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            epoch_testloss, running_corrects, len(test_loader.dataset),
            100. * epoch_testaccuracy))
            
        file1.close()

# Evaluating models on Actor wise split data

In [41]:
# Change to fit hardware

data_dir = '/scratch/kv942/ccm_proj/datadir_act/'

data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
class_names = image_datasets['test'].classes

test_dataset = ImageFolderWithPaths(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                       )

## 1) Alexnet

In [35]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/actor_wise_split/alexnet_40.pth'
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader)

Test set: Average loss: 2.4848, Accuracy: 2220/4450 (49.89%)



## 2) VGG16

In [42]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/actor_wise_split/VGG16_40.pth'
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader, class_names)

01-01-02-02-02-01-24-3.0.jpg	calm

01-01-07-02-02-01-23-3.0.jpg	disgust

01-01-07-01-02-01-23-2.55.jpg	disgust

01-01-06-01-01-02-23-3.0.jpg	sad

01-01-03-02-02-02-23-1.5.jpg	happy

01-01-04-01-02-02-23-1.95.jpg	surprised

01-02-05-01-02-01-24-3.6.jpg	angry

01-02-03-01-02-01-23-1.05.jpg	neutral

01-01-06-01-02-02-24-1.8.jpg	fearful

01-01-02-02-02-01-23-3.9.jpg	neutral

01-02-01-01-02-01-24-2.25.jpg	fearful

01-02-06-01-02-01-24-3.3.jpg	fearful

01-02-04-01-01-01-23-3.9.jpg	neutral

01-01-04-01-02-01-24-3.3.jpg	calm

01-02-05-01-02-01-23-2.55.jpg	disgust

01-01-06-02-01-01-23-1.2.jpg	fearful

01-01-03-01-02-01-23-2.1.jpg	happy

01-01-08-01-02-01-24-2.1.jpg	angry

01-01-07-01-01-02-23-1.95.jpg	disgust

01-01-01-01-02-02-24-3.0.jpg	angry

01-02-02-01-02-02-24-2.25.jpg	happy

01-01-05-02-01-01-23-1.8.jpg	angry

01-01-02-01-01-01-24-1.2.jpg	happy

01-02-02-01-01-02-24-2.7.jpg	fearful

01-02-01-01-02-02-24-3.75.jpg	fearful

01-01-08-01-01-02-24-1.5.jpg	fearful

01-02-05-01-01-02-24-2.55.jp

# Evaluating models on Random split data

In [49]:
# Change to fit hardware

data_dir = '/scratch/kv942/ccm_proj/datadir_rnd/'
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
class_names = image_datasets['test'].classes

test_dataset = ImageFolderWithPaths(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                       )


## 1) Alexnet

In [40]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/random_data_split/alexnet_40.pth'
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader)

Test set: Average loss: 0.0930, Accuracy: 4693/4837 (97.02%)



## 2) VGG16

In [51]:
PATH = '/scratch/sg5783/CCM_Project/final_folder/model_weights/random_data_split/VGG16_40.pth'
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 8)

model.load_state_dict(torch.load(PATH, map_location='cpu'))
test_performance(model, test_loader, class_names)

01-02-06-01-02-01-05-3.6.jpg	fearful

01-01-04-01-01-02-22-2.4.jpg	sad

01-01-03-01-02-01-21-2.25.jpg	happy

01-02-03-02-02-02-03-2.1.jpg	happy

01-01-02-02-02-01-01-2.25.jpg	calm

01-01-05-01-02-01-12-2.7.jpg	angry

01-01-05-01-02-02-15-3.45.jpg	angry

01-01-07-01-02-01-11-1.65.jpg	disgust

01-02-03-02-02-01-03-4.05.jpg	happy

01-01-07-02-01-02-01-3.15.jpg	disgust

01-02-06-02-01-01-14-3.45.jpg	fearful

01-01-02-01-01-01-24-3.45.jpg	calm

01-01-01-01-02-01-13-2.25.jpg	neutral

01-02-01-01-02-01-06-1.95.jpg	neutral

01-02-06-02-01-02-24-3.0.jpg	fearful

01-02-04-02-02-02-13-4.35.jpg	sad

01-02-06-01-01-01-08-1.05.jpg	sad

01-02-01-01-02-02-04-2.7.jpg	neutral

01-02-03-01-01-02-10-2.7.jpg	happy

01-01-05-01-01-01-22-1.5.jpg	angry

01-02-06-01-01-02-06-4.35.jpg	fearful

01-02-03-02-02-01-13-1.05.jpg	happy

01-01-03-02-01-01-19-2.7.jpg	happy

01-02-02-02-02-02-03-4.35.jpg	calm

01-02-02-01-02-01-06-1.5.jpg	calm

01-01-03-01-01-01-16-2.1.jpg	happy

01-02-03-02-01-01-16-3.15.jpg	happy

01-0